# 使用 Chroma 但不适用 LlamaIndex 的比较

结论：

- 不使用LlamaIndex的情况下，得分和Qdrant是一致的
- 问题出在LlamaIndex集成Chroma的代码上

## 准备

In [1]:
%%time
%%capture

!pip install chromadb

CPU times: user 8.72 ms, sys: 16.9 ms, total: 25.6 ms
Wall time: 4.6 s


In [2]:
%%time

documents = [
    {
        "name": "围城",
        "description": "主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。",
        "author": "钱钟书",
    },
    {
        "name": "故乡",
        "description": "小说讲述了主人公“我”（即鲁迅的化身）在阔别多年后回到故乡接母亲到城里生活的故事。在故乡，他遇到了童年的玩伴闰土和老仆人杨二嫂。通过与他们的交谈和观察，主人公感受到故乡的变化和人们生活的困苦。",
        "author": "鲁迅",
    },
    {
       "name": "阿Q正传",
        "description": "讲述了阿Q这个贫苦农民在中国封建社会中的悲惨生活。他虽然穷困潦倒，但心态自负，总是以精神胜利法来安慰自己，逃避现实的困境。然而，随着社会动荡和革命的到来，阿Q的命运变得更加悲惨，最终被误认为是革命党人而被处死。",
        "author": "鲁迅",
    }, 
]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.81 µs


In [3]:
%%time

# 嵌入模型

from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('/models/bge-small-zh-v1.5')

CPU times: user 2.29 s, sys: 391 ms, total: 2.68 s
Wall time: 2.42 s


In [4]:
encoder.get_sentence_embedding_dimension()

512

## 使用 Chroma

### 设置存储位置

In [5]:
%%time

import chromadb

collection_name="my_books"

chroma_client = chromadb.EphemeralClient() # 临时客户端，内存存储
chroma_collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"},
)

CPU times: user 509 ms, sys: 3.98 ms, total: 513 ms
Wall time: 512 ms


In [6]:
chroma_collection.count()

0

### 向集合传入数据

In [7]:
%%time

chroma_collection.add(
    documents=[doc["description"] for doc in documents],
    embeddings=[encoder.encode(doc["description"]).tolist() for doc in documents],
    metadatas=[{'name': doc['name'], 'author': doc['author']} for doc in documents],
    ids=[f"{i + 1}" for i in range(len(documents))],
)

chroma_collection.count()

CPU times: user 423 ms, sys: 71.3 ms, total: 495 ms
Wall time: 493 ms


3

### 查询

In [8]:
%%time

results=chroma_collection.query(
    query_embeddings=encoder.encode("方鸿渐").tolist(),
    n_results=3,
)

results

CPU times: user 12.7 ms, sys: 111 µs, total: 12.9 ms
Wall time: 11.9 ms


{'ids': [['1', '2', '3']],
 'distances': [[0.3904017210006714, 0.7029688358306885, 0.727487325668335]],
 'metadatas': [[{'author': '钱钟书', 'name': '围城'},
   {'author': '鲁迅', 'name': '故乡'},
   {'author': '鲁迅', 'name': '阿Q正传'}]],
 'embeddings': None,
 'documents': [['主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。',
   '小说讲述了主人公“我”（即鲁迅的化身）在阔别多年后回到故乡接母亲到城里生活的故事。在故乡，他遇到了童年的玩伴闰土和老仆人杨二嫂。通过与他们的交谈和观察，主人公感受到故乡的变化和人们生活的困苦。',
   '讲述了阿Q这个贫苦农民在中国封建社会中的悲惨生活。他虽然穷困潦倒，但心态自负，总是以精神胜利法来安慰自己，逃避现实的困境。然而，随着社会动荡和革命的到来，阿Q的命运变得更加悲惨，最终被误认为是革命党人而被处死。']],
 'uris': None,
 'data': None}

In [9]:
print(results['documents'][0], "score: ", 1-results['distances'][0][0])

['主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。', '小说讲述了主人公“我”（即鲁迅的化身）在阔别多年后回到故乡接母亲到城里生活的故事。在故乡，他遇到了童年的玩伴闰土和老仆人杨二嫂。通过与他们的交谈和观察，主人公感受到故乡的变化和人们生活的困苦。', '讲述了阿Q这个贫苦农民在中国封建社会中的悲惨生活。他虽然穷困潦倒，但心态自负，总是以精神胜利法来安慰自己，逃避现实的困境。然而，随着社会动荡和革命的到来，阿Q的命运变得更加悲惨，最终被误认为是革命党人而被处死。'] score:  0.6095982789993286


### 过滤条件

In [10]:
%%time

results=chroma_collection.query(
    query_embeddings=encoder.encode("方鸿渐").tolist(),
    n_results=3,
    where={"author": "钱钟书"},
)

results

CPU times: user 4.13 ms, sys: 4.03 ms, total: 8.16 ms
Wall time: 7.5 ms


{'ids': [['1']],
 'distances': [[0.3904017210006714]],
 'metadatas': [[{'author': '钱钟书', 'name': '围城'}]],
 'embeddings': None,
 'documents': [['主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。']],
 'uris': None,
 'data': None}

In [11]:
print(results['documents'][0], "score: ", 1-results['distances'][0][0])

['主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。'] score:  0.6095982789993286
